In [1]:
import os

import warnings 
warnings.filterwarnings('ignore')


# for data analytics
import pandas as pd
import numpy as np
from collections import Counter

# for visualizations
#import seaborn as sns
import matplotlib.pyplot as plt

# for data preparation
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

# imblean provides tools for us to deal with imbalanced class sizes
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTEENN

# # For entropy computation
# from pyitlib import discrete_random_variable as drv

from scipy import stats
import missingno
from sklearn import preprocessing

# TEXT EDA
# from wordcloud import WordCloud

# import spacy
# from spacy.lang.en.stop_words import STOP_WORDS
# from spacy.lang.en import English

#import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 100
#import matplotlib.gridspec as gridspec
#import seaborn as sns; sns.set()
import ast

from datetime import date, time, datetime
import calendar

import pandas as pd
import emoji
import regex as re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from deep_translator import GoogleTranslator


import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/natalie/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/natalie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def read_file():
    full_data = pd.DataFrame()
    for file in os.listdir('../scrape/restaurant-data'):
        if 'compiled' in file:
            data = pd.read_csv(f'../scrape/restaurant-data/{file}', index_col = 0)
            full_data = full_data.append(data)
    return full_data

In [3]:
data = read_file()
data = data.drop_duplicates()

## General counts

In [4]:
# Total Number of restaurant reviews
print(f'No. reviews: {len(data)}')
# Number of restaurants with reviews
num = len(data['url'].unique())
print(f'No. restaurants: {num}')

No. reviews: 28731
No. restaurants: 863


## Clean Cost Column 

In [5]:
data['cleaned_price'] = data['price'].apply(lambda x: int(x.split('/')[0].replace('~$','')) if '$' in x else np.nan)
data.head(3)

,url,name,neighbourhood,price,categories,review,user,date,cleaned_price
0,https://www.burpple.com/bedok-85-market?bp_ref...,85 Fengshan Centre,Bedok,~$5/pax,"['Hawker Food', 'Supper', 'Cheap & Good']","\nFish Ball Minced Meat Noodle\nFishball, meat...",Triffany Lim,21m ago,5.0
1,https://www.burpple.com/bedok-85-market?bp_ref...,85 Fengshan Centre,Bedok,~$5/pax,"['Hawker Food', 'Supper', 'Cheap & Good']","\nOrh lua\nThere are a couple of stores, but g...",Ally Tan,Jul 30 at 4:12pm,5.0
2,https://www.burpple.com/bedok-85-market?bp_ref...,85 Fengshan Centre,Bedok,~$5/pax,"['Hawker Food', 'Supper', 'Cheap & Good']",\nPeanut sauce was ace\nI love a good satay pe...,Ally Tan,Jul 30 at 4:10pm,5.0


## Clean Categories

In [6]:
category_mapping = {'Steak':['Western'], 
 'Cocktails':['Alcohol','Drinks'], 
 'Great View': ['Good Environment'], 
 'Sushi':['Japanese'], 
 'Ramen':['Japanese','Noodles'], 
 'Islandwide Delivery':['Convenient'], 
 'Craft Beer':['Drinks','Alcohol'], 
 '1 For 1 Deals':['Value'],
 'Teppanyaki':['Japanese'], 
 'Sustainable Dining on Beyond':['Sustainable'], 
 'Vegan friendly':['Sustainable'], 
 'Kopitiam':['Local Delights'], 
 'Char Kway Teow':['Local Delights','Noodles','Chinese'], 
 'Taiwanese':['Chinese'], 
 'Waffles':['Desserts'], 
 'Zi Char':['Local Delights','Chinese'], 
 'Fruit Tea':['Drinks'], 
 'Pasta':['Western','Noodles'], 
 'Vouchers':['Value'], 
 'Chirashi':['Japanese'], 
 'Bars':['Alcohol','Drinks'], 
 'Burpple Beyond Deals 💰':['Value'], 
 'BITES':['Bites'],
 'Burpple Guides':['Recommended'],
 'Michelin Guide Singapore 2018':['Recommended'],
'Grill & BBQ':['Western','BBQ'], 
'Bread & Pastries':['Desserts','High Tea', 'Breakfast & Brunch'], 
 'Vegetarian friendly':['Sustainable'], 
 'Cheap & Good':['Value'], 
 'Mediterranean':['European'], 
 'Middle Eastern': ['European'],
 'Michelin Guide Singapore 2017':['Recommended'], 
 'Hawker Food':['Local Delights'], 
 'Ice Cream & Yoghurt':['Desserts'], 
 'Cafes & Coffee':['Desserts','High Tea', 'Breakfast & Brunch'], 
 'Interesting':['Good Environment'],
'Dinner with Drinks':['Drinks'], 
'Bak Kut Teh':['Local Delights', 'Chinese'],
'Burgers':['Western'],
'Korean Desserts': ['Desserts','Korean'],
 'Vegetarian':['Sustainable'], 
 'Nasi Lemak':['South East Asian','Local Delights'], 
 'Salads':['Western'], 
'TAKEAWAY OPTION':['Convenient'],
'Delivery':['Convenient'], 
'Sandwiches':['High Tea', 'Breakfast & Brunch', 'Western'], 
'Pizza':['Western'],
'Vegan':['Sustainable'], 
'Dim Sum':['Chinese'], 
'Chicken Rice':['Local Delights','Chinese'], 
'Fried Chicken':['Korean', 'Western', 'Bites','Fast Food'], 
'Korean BBQ':['Korean','BBQ'],
'Filipino Local Delights':['Filipino'], 
'Cakes':['Desserts','High Tea'], 
'Michelin Guide Singapore 2019':['Recommended'],
 'Korean Fried Chicken':['Korean','Bites','Fast Food'], 
'Hot Pot': ['Chinese', 'Rainy Day Comforts'], 
'Soup': ['Rainy Day Comforts'],
'Late Night':['Supper'],
 'Bubble Tea':['Drinks'],
 'BEYOND': ['Recommended', 'Value'],
 'Argentinian':['European'],
 'Filipino Local Delights': ['South East Asian'],
 'Indonesian': ['South East Asian'],
 'Malay': ['South East Asian'],
 'Vietnamese':['South East Asian'],
 'Peranakan': ['South East Asian'],
 'Thai': ['South East Asian'],
 'Greek':['European'],
 'Russian': ['European'],
 'Turkish': ['European'],
 'Mexican': ['European'],
 'Spanish': ['European'],
 'Newly Opened': ['Novel'],
 'Hidden Gem':['Novel'],
 'French': ['European'],
 'Italian': ['European'],
 'Kid Friendly': ['Accessible'],
 'Pet-Friendly': ['Accessible'],
 'Good For Groups': ['Accessible']
 }

In [7]:
# find unique categories 
def find_unique_categories(category_col_name, data):
    all_categories = []
    for categories in data[category_col_name]:
        try:
            all_categories.extend(ast.literal_eval(categories))
        except:
            all_categories.extend(categories)
    print(f'Num of Categories:{len(set(all_categories))}')
    for i in set(all_categories):
        print(i)
    return list(set(all_categories))
    

In [8]:
find_unique_categories('categories', data)

Num of Categories:101
Hot Pot
Bread & Pastries
Kopitiam
Good For Groups
Japanese
Char Kway Teow
Michelin Guide Singapore 2019
Vegan friendly
TAKEAWAY OPTION
Vegetarian
BEYOND
Vietnamese
Supper
Taiwanese
Korean
Waffles
Sandwiches
Burpple Guides
Craft Beer
Islandwide Delivery
Vegetarian friendly
1 For 1 Deals
Mexican
Newly Opened
Vegan
Sushi
Bites
Late Night
BBQ
Soup
Russian
Michelin Guide Singapore 2018
Chirashi
Grill & BBQ
Malay
Kid Friendly
Korean BBQ
High Tea
Chicken Rice
Sustainable Dining on Beyond
Hawker Food
Bak Kut Teh
Italian
Halal
Breakfast & Brunch
Pasta
Spanish
Fast Food
Zi Char
Pizza
Michelin Guide Singapore 2017
Peranakan
Western
Burpple Beyond Deals 💰
Greek
Cheap & Good
Korean Fried Chicken
Dinner with Drinks
Nasi Lemak
Healthy
Mediterranean
Ice Cream & Yoghurt
Cocktails
Interesting
Teppanyaki
Middle Eastern
Local Delights
Salads
Fruit Tea
Cakes
Thai
Fine Dining
French
Desserts
Burgers
Steak
Chinese
Cafes & Coffee
Delivery
Buffets
Bubble Tea
Fried Chicken
Hidden Gem
Europ

['Hot Pot',
 'Bread & Pastries',
 'Kopitiam',
 'Good For Groups',
 'Japanese',
 'Char Kway Teow',
 'Michelin Guide Singapore 2019',
 'Vegan friendly',
 'TAKEAWAY OPTION',
 'Vegetarian',
 'BEYOND',
 'Vietnamese',
 'Supper',
 'Taiwanese',
 'Korean',
 'Waffles',
 'Sandwiches',
 'Burpple Guides',
 'Craft Beer',
 'Islandwide Delivery',
 'Vegetarian friendly',
 '1 For 1 Deals',
 'Mexican',
 'Newly Opened',
 'Vegan',
 'Sushi',
 'Bites',
 'Late Night',
 'BBQ',
 'Soup',
 'Russian',
 'Michelin Guide Singapore 2018',
 'Chirashi',
 'Grill & BBQ',
 'Malay',
 'Kid Friendly',
 'Korean BBQ',
 'High Tea',
 'Chicken Rice',
 'Sustainable Dining on Beyond',
 'Hawker Food',
 'Bak Kut Teh',
 'Italian',
 'Halal',
 'Breakfast & Brunch',
 'Pasta',
 'Spanish',
 'Fast Food',
 'Zi Char',
 'Pizza',
 'Michelin Guide Singapore 2017',
 'Peranakan',
 'Western',
 'Burpple Beyond Deals 💰',
 'Greek',
 'Cheap & Good',
 'Korean Fried Chicken',
 'Dinner with Drinks',
 'Nasi Lemak',
 'Healthy',
 'Mediterranean',
 'Ice Cream 

In [9]:
# map categories
all_new_categories = []
for idx, row in data.iterrows():
    categories = ast.literal_eval(row['categories'])
    new_categories = []
    for category in categories:
        if category.strip() in category_mapping.keys():
            new_categories.extend(category_mapping[category.strip()])
        else:
            new_categories.append(category)
    all_new_categories.append(new_categories)

data['cleaned_categories'] = all_new_categories

In [10]:
# find unique categories after mapping
list_cleaned_categories = find_unique_categories('cleaned_categories', data)

Num of Categories:32
Chinese
Western
Recommended
Buffets
Convenient
Bites
Japanese
Rainy Day Comforts
European
BBQ
Good Environment
Healthy
Indian
Drinks
South East Asian
Novel
Date Night
Supper
Local Delights
High Tea
Sustainable
Korean
Value
Seafood
Halal
Breakfast & Brunch
Noodles
Alcohol
Fast Food
Fine Dining
Accessible
Desserts


## Clean Text

In [11]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [12]:
#decontraction
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"didn\'t", "did not", phrase)
    phrase = re.sub(r"don\'t", "do not", phrase)
    phrase = re.sub(r"o\'clock", "clock", phrase)
    phrase = re.sub(r"couldn\'t", "could not", phrase)
    phrase = re.sub(r"that\'s", "that is", phrase)       
    phrase = re.sub(r"go-around", "go around", phrase)  
    # general
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    #phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [13]:
def clean_text(df):
    text_list = df['review']

    stopwords = nltk.corpus.stopwords.words('english')
    new_stopwords = ['address', 'note', 'tel', 'website', 'open', 'burpple']
    add_stopwords = set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
        "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
        'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
        'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
        'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
        'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
        'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
        'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
        'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
        'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
        's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
        've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
        "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
        "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
        'won', "won't", 'wouldn', "wouldn't"])
    add_stopwords_2 =  ['n','s','m','i','1','2','3','4','5','6','7','8','9','10','one','two',
        'it','in','ve','well','could','would','really','also','even',
        'alway','always','still','never','much','thing','yet',
        'said','asked','did','go','got','do','make','know','think','come','going',
        'put','went','seem','order','ordered','give','eat','make','get']
    
    stopwords.extend(new_stopwords)
    stopwords.extend(add_stopwords)
    stopwords.extend(add_stopwords_2)

    ### FOR SENTIMENT ANALYSIS< UNCOMMENT THIS
    # not_stopwords = {'no','nor','not'} 
    # stopwords = set([word for word in stopwords if word not in not_stopwords])

    lemmatizer = WordNetLemmatizer()
    ps = PorterStemmer()
    cleaned_text_list = []
    count = 0
    for text in text_list:

        # lower case
        text = text.lower()

        # demojize    
        text = emoji.demojize(text)

        # remove headers
        text = ' '.join(text.split('\n')[2:])

        # remove location (pushpin or location:)
        text = text.split('round_pushpin')[0]
        text = text.split('location:')[0]
    
        # clean punctuation
        text = re.sub(r'[^\w\s]', '', text)

        # tokenize
        tokens = word_tokenize(text)

        # remove stopwords and urls
        tokens = [word for word in tokens if word not in stopwords]
        tokens = [word for word in tokens if 'http' not in word]
        tokens = [word for word in tokens if 'www' not in word]

        # decontraction
        tokens = [decontracted(word) for word in tokens]

        # translate to english
        # text = GoogleTranslator(source='auto', target='en').translate(text)
    
        # POS tagging
        tokens = [nltk.pos_tag([word]) for word in tokens]

        # lemmatization
        tokens = [lemmatizer.lemmatize(word[0][0], get_wordnet_pos(word[0][1])) 
            if get_wordnet_pos(word[0][1])!=None else lemmatizer.lemmatize(word[0][0]) for word in tokens]
    
        # concatenate tokens back
        cleaned_text = " ".join(tokens)
        cleaned_text_list.append(cleaned_text)

        if count%1000 == 0:
            print(count)
        count+=1

    df['cleaned_text'] = cleaned_text_list
    
    return df

In [14]:
cleaned_df = clean_text(data)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [15]:
# replace empty strings with np.nan
cleaned_df = cleaned_df.replace(r'^\s*$', np.nan, regex=True)

## Check for null values

In [16]:
# Check if there is any null value
cleaned_df.isna().apply(pd.value_counts)

,url,name,neighbourhood,price,categories,review,user,date,cleaned_price,cleaned_categories,cleaned_text
False,28731.0,28731.0,28708,28731.0,28731.0,28731.0,28731.0,28731.0,28302,28731.0,28503
True,NaN,NaN,23,NaN,NaN,NaN,NaN,NaN,429,NaN,228


In [17]:
# 89 reviews have no neighbourhood. Thus, we drop these reviews.
cleaned_data = cleaned_df[~cleaned_df['neighbourhood'].isna()]

# 1781 reviews have no price. Thus, we drop these reviews.
cleaned_data = cleaned_data[~cleaned_data['cleaned_price'].isna()]

# 2 reviews have no review text. Thus, we drop these reviews.
cleaned_data = cleaned_data[~cleaned_data['cleaned_text'].isna()]

In [18]:
# Total Number of restaurant reviews
print(f'No. reviews: {len(cleaned_data)}')
# Number of restaurants with reviews
num = len(cleaned_data['url'].unique())
print(f'No. restaurants: {num}')

No. reviews: 28075
No. restaurants: 845


## Convert Cat Variables to Binary Variables

In [19]:
list_cleaned_categories = find_unique_categories('cleaned_categories', cleaned_data)
list_cleaned_categories

Num of Categories:32
Chinese
Western
Recommended
Buffets
Convenient
Bites
Japanese
Rainy Day Comforts
European
BBQ
Good Environment
Healthy
Indian
Drinks
South East Asian
Novel
Date Night
Supper
Local Delights
High Tea
Sustainable
Korean
Value
Seafood
Halal
Breakfast & Brunch
Noodles
Alcohol
Fast Food
Fine Dining
Accessible
Desserts


['Chinese',
 'Western',
 'Recommended',
 'Buffets',
 'Convenient',
 'Bites',
 'Japanese',
 'Rainy Day Comforts',
 'European',
 'BBQ',
 'Good Environment',
 'Healthy',
 'Indian',
 'Drinks',
 'South East Asian',
 'Novel',
 'Date Night',
 'Supper',
 'Local Delights',
 'High Tea',
 'Sustainable',
 'Korean',
 'Value',
 'Seafood',
 'Halal',
 'Breakfast & Brunch',
 'Noodles',
 'Alcohol',
 'Fast Food',
 'Fine Dining',
 'Accessible',
 'Desserts']

In [20]:
# append empty columns with category names
cleaned_data = cleaned_data.reindex(cleaned_data.columns.tolist() + list_cleaned_categories, axis=1)

In [21]:
for category in list_cleaned_categories:
    print(category)
    cleaned_data[category] = cleaned_data['cleaned_categories'].apply(lambda x: 1 if category in x else 0)

Chinese
Western
Recommended
Buffets
Convenient
Bites
Japanese
Rainy Day Comforts
European
BBQ
Good Environment
Healthy
Indian
Drinks
South East Asian
Novel
Date Night
Supper
Local Delights
High Tea
Sustainable
Korean
Value
Seafood
Halal
Breakfast & Brunch
Noodles
Alcohol
Fast Food
Fine Dining
Accessible
Desserts


## Mapping Neighbourhoods to Regions

In [22]:
neighbourhood_to_region_mapping = pd.read_excel('neighbourhood_to_region_mapping.xlsx')
neighbourhood_to_region_mapping['neighbourhood'] = neighbourhood_to_region_mapping['neighbourhood'].apply(lambda x: x.replace('\xa0',''))
neighbourhood_to_region_mapping = neighbourhood_to_region_mapping.set_index('neighbourhood').T.to_dict('records')[0]

FileNotFoundError: [Errno 2] No such file or directory: 'neighbourhood_to_region_mapping.xlsx'

In [ ]:
cleaned_data['region'] = cleaned_data['neighbourhood'].apply(lambda x: neighbourhood_to_region_mapping[x])

In [ ]:
cleaned_data

In [ ]:
cleaned_data.to_csv('cleaned_restaurant_reviews.csv')